# Time Series Analysis of Remote Sensing Data

This notebook demonstrates how to perform time series analysis on a stack of remote sensing rasters using `rasterio`, `numpy`, and `pandas` in Python. Time series analysis is useful for monitoring changes in land cover, vegetation, or other phenomena over time.

## Prerequisites
- Install required libraries: `rasterio`, `numpy`, `pandas`, `matplotlib` (listed in `requirements.txt`).
- A set of GeoTIFF files representing a time series (e.g., `raster_2023_01.tif`, `raster_2023_02.tif`, etc.). Replace file paths with your own data.

## Learning Objectives
- Load and stack multiple rasters into a time series.
- Compute temporal statistics (e.g., mean, trend) for a specific band.
- Visualize time series data and trends.

In [ ]:
# Import required libraries
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from scipy.stats import linregress

## Step 1: Load Time Series Rasters

Load a series of raster files and stack them into a 4D array (time, bands, height, width).

In [ ]:
# Define directory and pattern for raster files
raster_dir = 'time_series_data/'
raster_files = sorted(glob.glob(raster_dir + 'raster_*.tif'))  # Replace with your file pattern

# Load rasters
raster_stack = []
dates = []
for file in raster_files:
    with rasterio.open(file) as src:
        raster_stack.append(src.read())
        profile = src.profile
    # Extract date from filename (assumes format like raster_2023_01.tif)
    date = file.split('_')[-1].replace('.tif', '')
    dates.append(pd.to_datetime(date, format='%Y_%m'))

# Convert to numpy array
raster_stack = np.stack(raster_stack, axis=0)  # Shape: (time, bands, height, width)

# Print basic information
print(f'Time series shape: {raster_stack.shape}')
print(f'Time points: {dates}')

## Step 2: Compute Temporal Statistics

Calculate the mean and trend for a specific band (e.g., band 1) across the time series.

In [ ]:
# Select band 1 (index 0) for analysis
band_idx = 0
band_data = raster_stack[:, band_idx, :, :]

# Compute temporal mean
temporal_mean = np.mean(band_data, axis=0)

# Compute linear trend (slope) for each pixel
time_indices = np.arange(len(dates))
trend = np.zeros((band_data.shape[1], band_data.shape[2]), dtype=np.float32)
for i in range(band_data.shape[1]):
    for j in range(band_data.shape[2]):
        pixel_ts = band_data[:, i, j]
        if np.any(np.isnan(pixel_ts)):
            trend[i, j] = np.nan
        else:
            slope, _, _, _, _ = linregress(time_indices, pixel_ts)
            trend[i, j] = slope

# Visualize temporal mean
plt.figure(figsize=(8, 8))
plt.imshow(temporal_mean, cmap='viridis')
plt.colorbar(label='Mean Value')
plt.title('Temporal Mean of Band 1')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Visualize trend
plt.figure(figsize=(8, 8))
plt.imshow(trend, cmap='RdBu', vmin=-np.nanmax(np.abs(trend)), vmax=np.nanmax(np.abs(trend)))
plt.colorbar(label='Trend (Slope)')
plt.title('Temporal Trend of Band 1')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 3: Time Series Plot for a Pixel

Extract and plot the time series for a specific pixel.

In [ ]:
# Select a pixel (e.g., center of the image)
pixel_i, pixel_j = band_data.shape[1] // 2, band_data.shape[2] // 2
pixel_ts = band_data[:, pixel_i, pixel_j]

# Plot time series
plt.figure(figsize=(8, 6))
plt.plot(dates, pixel_ts, marker='o')
plt.title(f'Time Series for Pixel ({pixel_i}, {pixel_j})')
plt.xlabel('Date')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## Step 4: Save Temporal Statistics

Save the temporal mean and trend as GeoTIFF files.

In [ ]:
# Update profile for single-band output
output_profile = profile.copy()
output_profile.update(count=1, dtype=rasterio.float32)

# Save temporal mean
with rasterio.open('temporal_mean.tif', 'w', **output_profile) as dst:
    dst.write(temporal_mean, 1)

# Save trend
with rasterio.open('temporal_trend.tif', 'w', **output_profile) as dst:
    dst.write(trend, 1)

print('Temporal mean saved to: temporal_mean.tif')
print('Temporal trend saved to: temporal_trend.tif')

## Next Steps

- Replace `raster_dir` with the path to your time series raster files.
- Adjust the date extraction logic to match your file naming convention.
- Compute additional statistics (e.g., standard deviation, seasonal decomposition).
- Proceed to the next notebook (`18_change_detection.ipynb`) for change detection analysis.

## Notes
- Ensure all rasters have the same CRS, resolution, and dimensions.
- Handle no-data values appropriately during calculations.
- See `docs/installation.md` for troubleshooting library installation.